In [1]:
import dropbox
import urllib.request, json 
from collections import defaultdict
from collections import OrderedDict
import datetime as DT
import ast
from __future__ import print_function
from __future__ import division
import collections
import nltk
import numpy as np
from string import punctuation
import csv
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

# Helper libraries
from w266_common import utils, vocabulary

########################### DLEE IMPORTS ##################################

# Import pandas and tensorflow
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1.10"))

# Import model
import models

# Import misc
from importlib import reload
import os, sys, re, json, time, datetime, shutil
import itertools
from w266_common import patched_numpy_io

/home/felixtsui/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [80]:
##Download the pre-processed tweets from dropbox

access_token = 'p3Ze8FpdRTAAAAAAAAAADb9hCQ8aXXYU3A3gGM1HEXyeMrs8KI2SrA71KDmmCXte'
dbx = dropbox.Dropbox(access_token)
metadata, res = dbx.files_download(path="/Inputdata2.txt")


access_token2 = 'm_fKi8mWZ0AAAAAAAAAADlfzTm37f1y45k92Xpiw1B6mjT3dQqzNvKgpIUSR17uq'
dbx2 = dropbox.Dropbox(access_token2)

#Write the combined file into the drive
with open("InputX.txt", "wb") as f:
    metadata, res = dbx.files_download(path="/Inputdata2.txt")
    metadata2, res2 = dbx2.files_download(path="/Inputdata.txt")
    f.write(res.content)
    f.write(res2.content)

In [2]:
##Translate date of the tweet to quarter reference
def Quarterlookup(date):
    if date>=DT.datetime(2018,10,1):
        return '18Q4'
    elif date>=DT.datetime(2018,7,1):
        return '18Q3'
    elif date>=DT.datetime(2018,4,1):
        return '18Q2'
    elif date>=DT.datetime(2018,1,1):
        return '18Q1'
    elif date>=DT.datetime(2017,10,1):
        return '17Q4'
    elif date>=DT.datetime(2017,7,1):
        return '17Q3'
    elif date>=DT.datetime(2017,4,1):
        return '17Q2'
    elif date>=DT.datetime(2017,1,1):
        return '17Q1'
    elif date>=DT.datetime(2016,10,1):
        return '16Q4'
    elif date>=DT.datetime(2016,7,1):
        return '16Q3'
    elif date>=DT.datetime(2016,4,1):
        return '16Q2'
    elif date>=DT.datetime(2016,1,1):
        return '16Q1'
    elif date>=DT.datetime(2015,10,1):
        return '15Q4'
    elif date>=DT.datetime(2015,7,1):
        return '15Q3'
    elif date>=DT.datetime(2015,4,1):
        return '15Q2'
    else:
        return '15Q1'

In [3]:
##Reading the Input X file and process it
d=defaultdict(list)

with open('InputX.txt') as f:
    for line in f:
        count=0
        temp =-1
        k,v=line.split(':',1)
        while True:
            
            oldtemp=temp
            temp=v.find(']]',temp+1)
            if count==0:
                d[k].append(v[oldtemp+2:temp+2]) 
            else:
                d[k].append(v[oldtemp+4:temp+2])
           
             
            count+=1
            if temp == -1: break
#             if count == 500: break  # initially limit for time sake
    

In [4]:
#Read from the vocab file pre prepared
with open('vocab.csv', 'r') as f:
    reader = csv.reader(f,delimiter=';')
    allword = set(list(reader)[0])

In [5]:
allword=sorted(allword)
vocab = vocabulary.Vocabulary(allword, size=None)  # size=None means unlimited
print("Vocabulary size: {:,}".format(vocab.size))

Vocabulary size: 5,681


In [6]:
def pad_np_array(example_ids, max_len=50, pad_id=0):

    arr = np.full([len(example_ids), max_len], pad_id, dtype=np.int32)
    ns = np.zeros([len(example_ids)], dtype=np.int32)
    cpy_len = min(len(example_ids), max_len)

    arr = example_ids[:cpy_len]
    ns = cpy_len
    return arr, ns

In [7]:
#Dictionary to collect the tweet detail and the length of each tweet
InputX=defaultdict(dict)
InputN=defaultdict(dict)

In [8]:
#Set of all the stocks of interest
trainstock =['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger','WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty',"Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']

#trainstock =['CVSHealth', 'RiteAid']

In [9]:
##Converting the string of data into dictionary

#trainstock =['CVSHealth', 'RiteAid']
for key,value in d.items():
    Inputx=defaultdict(list)
    Inputn=defaultdict(list)
    if key in trainstock:
        for value2 in value:
        
            temp=value2.find('), ')
            date=value2[14:temp+3]
            try:
                year=int(date[1:5])
                month=int(date[date.find(',')+2:date.find(',',date.find(',')+1)])
                day=int(date[date.find(')')-2:date.find(')')])
            
                sentence, length = pad_np_array (vocab.words_to_ids(ast.literal_eval(value2[temp+3:-1])))
            
                Inputx[Quarterlookup(DT.datetime(year,month,day))].append(sentence)
                Inputn[Quarterlookup(DT.datetime(year,month,day))].append(length)
            except:
                pass

    InputX[key]=OrderedDict(sorted(Inputx.items()))
    InputN[key]=OrderedDict(sorted(Inputn.items()))

In [10]:
##List of stockname and relevant stock code
l = [("CVSHealth", "CVS"),
("RiteAid", "RAD"),
("Conn's", "CONN"),
("Casey's", "CASY"),
("Kroger", "KR"),
("WeisMarkets", "WMK"),
("HavertyFurniture", "HVT"),
("Kirkland's", "KIRK"),
("Pier1Imports","PIR"),
("BuildersFirstSource","BLDR"),
("HuttigBuildingProducts", "HBP"),
("LumberLiquidators", "LL"),
("TransWorldEntertainment", "TWMC"),
("MarineMax", "HZO"),
("Medifast", "MED"),
("SallyBeauty", "SBH"),
("Sotheby's", "BID"),
("StarGasPartners", "SGU"),
("SuburbanPropanePartners", "SPH"),
("TitanMachinery", "TITN"),
("TractorSupply","TSCO"),
("TravelCentersofAmerica", "TA"),
("Big5SportingGoods", "BGFV"),
("Dick'sSportingGoods", "DKS"),
("HibbettSports", "HIBB"),
("Build-A-BearWorkshop", "BBW")]
     
all_stock = {}
[all_stock.update({k:v}) for k,v in l]
print(all_stock)

{'CVSHealth': 'CVS', 'RiteAid': 'RAD', "Conn's": 'CONN', "Casey's": 'CASY', 'Kroger': 'KR', 'WeisMarkets': 'WMK', 'HavertyFurniture': 'HVT', "Kirkland's": 'KIRK', 'Pier1Imports': 'PIR', 'BuildersFirstSource': 'BLDR', 'HuttigBuildingProducts': 'HBP', 'LumberLiquidators': 'LL', 'TransWorldEntertainment': 'TWMC', 'MarineMax': 'HZO', 'Medifast': 'MED', 'SallyBeauty': 'SBH', "Sotheby's": 'BID', 'StarGasPartners': 'SGU', 'SuburbanPropanePartners': 'SPH', 'TitanMachinery': 'TITN', 'TractorSupply': 'TSCO', 'TravelCentersofAmerica': 'TA', 'Big5SportingGoods': 'BGFV', "Dick'sSportingGoods": 'DKS', 'HibbettSports': 'HIBB', 'Build-A-BearWorkshop': 'BBW'}


In [11]:
##Read the quarterly return detail from edgaronline and convert into library
top="http://datafied.api.edgar-online.com/v2/corefinancials/qtr?primarysymbols="
bottom="&appkey=asc97xrhkyu4959aptu76zxj"
Outputy =defaultdict(dict)
for stockname,stock in all_stock.items():
    check=top+stock+bottom
    with urllib.request.urlopen(check) as url:
        data = json.loads(url.read().decode())
    count1=0
    count2=0
    end_date=[]
    revenue_change=[]
    last_rev=0

    for key, value in data.items():
        
        for key2, value2 in value.items():          
        
            if count1!=0:  
                for i in reversed(value2):
                  
                    for j in i["values"]:
                     
                        if count2!=0:   
                            if j["field"] =="periodenddate":
                                end_date.append(Quarterlookup(DT.datetime.strptime(j['value'], '%m/%d/%Y')))
                       
                            ##Convert revenue into the y_label of 0 for negative growth and 1 for positive growth
                            if j["field"] =="totalrevenue":
                                revenue_cal =round(float(j['value'])/float(last_rev) - 1,3)
                                if revenue_cal>0:
                                    revenue_change.append(1)
                                else:
                                    revenue_change.append(0)
                        else:
                            
                            if j["field"] =="totalrevenue":      
                                last_rev=j['value']
                              
                    count2+=1
            count1+=1
            
    ##Output will be a dictionary with stockname
    Outputy[stockname]["period"]=end_date
    Outputy[stockname]["revenuechange"]=revenue_change

In [21]:
with open('Outputy.txt', 'w') as f:
    print(Outputy, file=f)

In [12]:
train_x_init = []
train_n_init = []
train_y_init = []
dev_x_init = []
dev_n_init = []
dev_y_init = []
test_x_init = []
test_n_init = []
test_y_init = []


In [13]:
trainquarter=['15Q1','15Q2','15Q3','15Q4','16Q1','16Q2','16Q3','16Q4','17Q1','17Q2','17Q3','17Q4','18Q1']

for i in trainstock:
    for j in range(len(trainquarter)):
        try:
                train_x_init.append(InputX [i][trainquarter[j]])
                train_n_init.append(InputN [i][trainquarter[j]])
                length=len(InputN [i][trainquarter[j]])           
                train_y_init.append([Outputy [i]['revenuechange'][j]]*length)
      
        except:
            pass



In [14]:
#teststock =['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger','WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty',"Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']
teststock =['CVSHealth']
testquarter=['18Q2','18Q3']

for i in teststock:
    for j in range(len(testquarter)):
        try:
            if testquarter[j]=='18Q3':
                test_x_init.append(InputX [i][testquarter[j]])
                test_n_init.append(InputN [i][testquarter[j]])
                length=len(InputN [i][testquarter[j]])
                test_y_init.append([Outputy [i]['revenuechange'][j]]*length)   
            
            elif testquarter[j]=='18Q2':
                dev_x_init.append(InputX [i][testquarter[j]])
                dev_n_init.append(InputN [i][testquarter[j]])
                length=len(InputN [i][testquarter[j]])
                dev_y_init.append([Outputy [i]['revenuechange'][j]]*length)     
               
      
        except:
            pass

In [246]:
print(len(train_x_init))
print(len(dev_x_init))

26
1


In [15]:

train_x_init2 = []
train_n_init2 = []
train_y_init2 = []
dev_x_init2 = []
dev_n_init2 = []
dev_y_init2 = []
test_x_init2 = []
test_n_init2 = []
test_y_init2 = []

for i in range(len(train_x_init)):
    for j in range(len(train_x_init[i])):
        train_x_init2.append(train_x_init[i][j])
        train_n_init2.append(train_n_init[i][j])
        train_y_init2.append(train_y_init[i][j])
        
for i in range(len(dev_x_init)):
    for j in range(len(dev_x_init[i])):
        dev_x_init2.append(dev_x_init[i][j])
        dev_n_init2.append(dev_n_init[i][j])
        dev_y_init2.append(dev_y_init[i][j])
        
for i in range(len(test_x_init)):
    for j in range(len(test_x_init[i])):
        test_x_init2.append(test_x_init[i][j])
        test_n_init2.append(test_n_init[i][j])
        test_y_init2.append(test_y_init[i][j])



# NEURAL BAG OF WORDS MODEL #

In [ ]:
for i in range(10):

    print(train_x_init2[i])
    print(test_x_init2[i])
    print(dev_x_init2[i])


In [16]:
def pad_np_array2(example_ids, max_len=50, pad_id=0):

    arr = np.full([len(example_ids), max_len], pad_id, dtype=np.int32)
    ns = np.zeros([len(example_ids)], dtype=np.int32)
    for i, ids in enumerate(example_ids):
        cpy_len = min(len(ids), max_len)
        arr[i,:cpy_len] = ids[:cpy_len]
        ns[i] = cpy_len
    return arr ##I HAD TO REMOVE NS TO MAKE IT WORK< PLEASE TAKE A LOOK IF IT HAS ANY IMPACT

In [263]:
print(train_x[0][0])

1478


In [249]:
#print(len(train_x_init[3][4]))
#print(train_x_init[3][4])

In [19]:
train_x = pad_np_array2(train_x_init2, max_len = 50, pad_id=0)
train_n = np.array(train_n_init2)
train_y = np.array(train_y_init2)

test_x = pad_np_array2(test_x_init2, max_len = 50, pad_id=0)
test_n = np.array(test_n_init2)
test_y = np.array(test_y_init2)

dev_x = pad_np_array2(dev_x_init2, max_len = 50, pad_id=0)
dev_n = np.array(dev_n_init2)
dev_y = np.array(dev_y_init2)

In [20]:
print(len(train_x))


print(train_x)

print(type(train_x))
print(train_x.shape)


6692652
[[1478 5673 4936 ...    0    0    0]
 [ 161 5592 3437 ...    0    0    0]
 [1295 2249    2 ...    2   62 2258]
 ...
 [1417 2659 2526 ...    0    0    0]
 [   2  833   27 ...    0    0    0]
 [   2  833   27 ...    0    0    0]]
<class 'numpy.ndarray'>
(6692652, 50)


In [137]:
#train_x = np.array(train_x[0])
#dev_x = np.array(dev_x[0])
#test_x = np.array(test_x[0])

In [220]:
print(len(train_x))
print(len(train_x[0]))
print(len(dev_x))
print(len(dev_x))
print(len(test_x))
print(len(test_x[0]))

6741983
50
0
0
0


IndexError: index 0 is out of bounds for axis 0 with size 0

In [21]:
reload(models)

x, ns, y = train_x, train_n, train_y
batch_size = 25

# Specify model hyperparameters as used by model_fn
model_params = dict(V=vocab.size, embed_dim=50, hidden_dims=[25], num_classes=2,
                    encoder_type='bow',
                    lr=0.1, optimizer='adagrad', beta=0.01)
model_fn = models.classifier_model_fn

total_batches = 0
total_examples = 0
total_loss = 0
loss_ema = np.log(2)  # track exponential-moving-average of loss
ema_decay = np.exp(-1/10)  # decay parameter for moving average = np.exp(-1/history_length)
with tf.Graph().as_default(), tf.Session() as sess:
    ##
    # Construct the graph here. No session.run calls - just wiring up Tensors.
    ##
    # Add placeholders so we can feed in data.
    x_ph_  = tf.placeholder(tf.int32, shape=[None, x.shape[1]])  # [batch_size, max_len]
    ns_ph_ = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    y_ph_  = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    
    # Construct the graph using model_fn
    features = {"ids": x_ph_, "ns": ns_ph_}  # note that values are Tensors
    estimator_spec = model_fn(features, labels=y_ph_, mode=tf.estimator.ModeKeys.TRAIN,
                              params=model_params)
    loss_     = estimator_spec.loss
    train_op_ = estimator_spec.train_op
    
    ##
    # Done constructing the graph, now we can make session.run calls.
    ##
    sess.run(tf.global_variables_initializer())
    
    # Run a single epoch
    t0 = time.time()
    for (bx, bns, by) in utils.multi_batch_generator(batch_size, x, ns, y):
        # feed NumPy arrays into the placeholder Tensors
        feed_dict = {x_ph_: bx, ns_ph_: bns, y_ph_: by}
        batch_loss, _ = sess.run([loss_, train_op_], feed_dict=feed_dict)
        
        # Compute some statistics
        total_batches += 1
        total_examples += len(bx)
        total_loss += batch_loss * len(bx)  # re-scale, since batch loss is mean
        # Compute moving average to smooth out noisy per-batch loss
        loss_ema = ema_decay * loss_ema + (1 - ema_decay) * batch_loss
        
        if (total_batches % 125 == 0):
            print("{:5,} examples, moving-average loss {:.2f}".format(total_examples, 
                                                                      loss_ema))    
    print("Completed one epoch in {:s}".format(utils.pretty_timedelta(since=t0)))

  625 examples, moving-average loss 0.56
1,250 examples, moving-average loss 0.36
1,875 examples, moving-average loss 0.28
2,500 examples, moving-average loss 0.23
3,125 examples, moving-average loss 0.20
3,750 examples, moving-average loss 0.17
4,375 examples, moving-average loss 0.22
5,000 examples, moving-average loss 0.15
5,625 examples, moving-average loss 0.12
6,250 examples, moving-average loss 0.11
6,875 examples, moving-average loss 0.10
7,500 examples, moving-average loss 0.09
8,125 examples, moving-average loss 0.08
8,750 examples, moving-average loss 0.08
9,375 examples, moving-average loss 0.07
10,000 examples, moving-average loss 0.07
10,625 examples, moving-average loss 0.06
11,250 examples, moving-average loss 0.06
11,875 examples, moving-average loss 0.06
12,500 examples, moving-average loss 0.05
13,125 examples, moving-average loss 0.05
13,750 examples, moving-average loss 0.05
14,375 examples, moving-average loss 0.05
15,000 examples, moving-average loss 0.05
15,625 

123,125 examples, moving-average loss 0.04
123,750 examples, moving-average loss 0.04
124,375 examples, moving-average loss 0.04
125,000 examples, moving-average loss 0.04
125,625 examples, moving-average loss 0.04
126,250 examples, moving-average loss 0.04
126,875 examples, moving-average loss 0.04
127,500 examples, moving-average loss 0.04
128,125 examples, moving-average loss 0.04
128,750 examples, moving-average loss 0.04
129,375 examples, moving-average loss 0.04
130,000 examples, moving-average loss 0.04
130,625 examples, moving-average loss 0.04
131,250 examples, moving-average loss 0.04
131,875 examples, moving-average loss 0.27
132,500 examples, moving-average loss 0.09
133,125 examples, moving-average loss 0.05
133,750 examples, moving-average loss 0.05
134,375 examples, moving-average loss 0.05
135,000 examples, moving-average loss 0.04
135,625 examples, moving-average loss 0.04
136,250 examples, moving-average loss 0.04
136,875 examples, moving-average loss 0.04
137,500 exa

242,500 examples, moving-average loss 0.04
243,125 examples, moving-average loss 0.04
243,750 examples, moving-average loss 0.04
244,375 examples, moving-average loss 0.04
245,000 examples, moving-average loss 0.04
245,625 examples, moving-average loss 0.04
246,250 examples, moving-average loss 0.04
246,875 examples, moving-average loss 0.04
247,500 examples, moving-average loss 0.03
248,125 examples, moving-average loss 0.03
248,750 examples, moving-average loss 0.03
249,375 examples, moving-average loss 0.03
250,000 examples, moving-average loss 0.03
250,625 examples, moving-average loss 0.03
251,250 examples, moving-average loss 0.03
251,875 examples, moving-average loss 0.03
252,500 examples, moving-average loss 0.03
253,125 examples, moving-average loss 0.03
253,750 examples, moving-average loss 0.03
254,375 examples, moving-average loss 0.03
255,000 examples, moving-average loss 0.03
255,625 examples, moving-average loss 0.03
256,250 examples, moving-average loss 0.03
256,875 exa

361,875 examples, moving-average loss 0.04
362,500 examples, moving-average loss 0.04
363,125 examples, moving-average loss 0.04
363,750 examples, moving-average loss 0.04
364,375 examples, moving-average loss 0.04
365,000 examples, moving-average loss 0.04
365,625 examples, moving-average loss 0.04
366,250 examples, moving-average loss 0.04
366,875 examples, moving-average loss 0.04
367,500 examples, moving-average loss 0.04
368,125 examples, moving-average loss 0.04
368,750 examples, moving-average loss 0.04
369,375 examples, moving-average loss 0.04
370,000 examples, moving-average loss 0.04
370,625 examples, moving-average loss 0.04
371,250 examples, moving-average loss 0.04
371,875 examples, moving-average loss 0.04
372,500 examples, moving-average loss 0.04
373,125 examples, moving-average loss 0.04
373,750 examples, moving-average loss 0.04
374,375 examples, moving-average loss 0.04
375,000 examples, moving-average loss 0.04
375,625 examples, moving-average loss 0.04
376,250 exa

481,875 examples, moving-average loss 0.04
482,500 examples, moving-average loss 0.04
483,125 examples, moving-average loss 0.04
483,750 examples, moving-average loss 0.04
484,375 examples, moving-average loss 0.04
485,000 examples, moving-average loss 0.04
485,625 examples, moving-average loss 0.04
486,250 examples, moving-average loss 0.04
486,875 examples, moving-average loss 0.04
487,500 examples, moving-average loss 0.04
488,125 examples, moving-average loss 0.04
488,750 examples, moving-average loss 0.04
489,375 examples, moving-average loss 0.04
490,000 examples, moving-average loss 0.04
490,625 examples, moving-average loss 0.04
491,250 examples, moving-average loss 0.04
491,875 examples, moving-average loss 0.04
492,500 examples, moving-average loss 0.04
493,125 examples, moving-average loss 0.04
493,750 examples, moving-average loss 0.04
494,375 examples, moving-average loss 0.04
495,000 examples, moving-average loss 0.04
495,625 examples, moving-average loss 0.04
496,250 exa

...

6,131,250 examples, moving-average loss 0.04
6,131,875 examples, moving-average loss 0.04
6,132,500 examples, moving-average loss 0.04
6,133,125 examples, moving-average loss 0.04
6,133,750 examples, moving-average loss 0.04
6,134,375 examples, moving-average loss 0.04
6,135,000 examples, moving-average loss 0.04
6,135,625 examples, moving-average loss 0.04
6,136,250 examples, moving-average loss 0.04
6,136,875 examples, moving-average loss 0.04
6,137,500 examples, moving-average loss 0.04
6,138,125 examples, moving-average loss 0.04
6,138,750 examples, moving-average loss 0.04
6,139,375 examples, moving-average loss 0.04
6,140,000 examples, moving-average loss 0.04
6,140,625 examples, moving-average loss 0.04
6,141,250 examples, moving-average loss 0.04
6,141,875 examples, moving-average loss 0.04
6,142,500 examples, moving-average loss 0.04
6,143,125 examples, moving-average loss 0.04
6,143,750 examples, moving-average loss 0.04
6,144,375 examples, moving-average loss 0.04
6,145,000 

6,246,250 examples, moving-average loss 0.04
6,246,875 examples, moving-average loss 0.04
6,247,500 examples, moving-average loss 0.04
6,248,125 examples, moving-average loss 0.04
6,248,750 examples, moving-average loss 0.04
6,249,375 examples, moving-average loss 0.04
6,250,000 examples, moving-average loss 0.04
6,250,625 examples, moving-average loss 0.25
6,251,250 examples, moving-average loss 0.09
6,251,875 examples, moving-average loss 0.07
6,252,500 examples, moving-average loss 0.06
6,253,125 examples, moving-average loss 0.06
6,253,750 examples, moving-average loss 0.05
6,254,375 examples, moving-average loss 0.05
6,255,000 examples, moving-average loss 0.05
6,255,625 examples, moving-average loss 0.05
6,256,250 examples, moving-average loss 0.05
6,256,875 examples, moving-average loss 0.05
6,257,500 examples, moving-average loss 0.05
6,258,125 examples, moving-average loss 0.05
6,258,750 examples, moving-average loss 0.05
6,259,375 examples, moving-average loss 0.05
6,260,000 

6,361,875 examples, moving-average loss 0.05
6,362,500 examples, moving-average loss 0.05
6,363,125 examples, moving-average loss 0.05
6,363,750 examples, moving-average loss 0.05
6,364,375 examples, moving-average loss 0.05
6,365,000 examples, moving-average loss 0.05
6,365,625 examples, moving-average loss 0.05
6,366,250 examples, moving-average loss 0.05
6,366,875 examples, moving-average loss 0.05
6,367,500 examples, moving-average loss 0.05
6,368,125 examples, moving-average loss 0.05
6,368,750 examples, moving-average loss 0.05
6,369,375 examples, moving-average loss 0.05
6,370,000 examples, moving-average loss 0.05
6,370,625 examples, moving-average loss 0.05
6,371,250 examples, moving-average loss 0.05
6,371,875 examples, moving-average loss 0.05
6,372,500 examples, moving-average loss 0.05
6,373,125 examples, moving-average loss 0.05
6,373,750 examples, moving-average loss 0.05
6,374,375 examples, moving-average loss 0.05
6,375,000 examples, moving-average loss 0.05
6,375,625 

6,476,250 examples, moving-average loss 0.04
6,476,875 examples, moving-average loss 0.04
6,477,500 examples, moving-average loss 0.04
6,478,125 examples, moving-average loss 0.04
6,478,750 examples, moving-average loss 0.04
6,479,375 examples, moving-average loss 0.04
6,480,000 examples, moving-average loss 0.30
6,480,625 examples, moving-average loss 0.09
6,481,250 examples, moving-average loss 0.06
6,481,875 examples, moving-average loss 0.05
6,482,500 examples, moving-average loss 0.05
6,483,125 examples, moving-average loss 0.05
6,483,750 examples, moving-average loss 0.05
6,484,375 examples, moving-average loss 0.05
6,485,000 examples, moving-average loss 0.04
6,485,625 examples, moving-average loss 0.04
6,486,250 examples, moving-average loss 0.04
6,486,875 examples, moving-average loss 0.04
6,487,500 examples, moving-average loss 0.04
6,488,125 examples, moving-average loss 0.04
6,488,750 examples, moving-average loss 0.04
6,489,375 examples, moving-average loss 0.04
6,490,000 

6,591,875 examples, moving-average loss 0.04
6,592,500 examples, moving-average loss 0.04
6,593,125 examples, moving-average loss 0.04
6,593,750 examples, moving-average loss 0.04
6,594,375 examples, moving-average loss 0.04
6,595,000 examples, moving-average loss 0.04
6,595,625 examples, moving-average loss 0.04
6,596,250 examples, moving-average loss 0.04
6,596,875 examples, moving-average loss 0.04
6,597,500 examples, moving-average loss 0.04
6,598,125 examples, moving-average loss 0.04
6,598,750 examples, moving-average loss 0.04
6,599,375 examples, moving-average loss 0.04
6,600,000 examples, moving-average loss 0.04
6,600,625 examples, moving-average loss 0.04
6,601,250 examples, moving-average loss 0.04
6,601,875 examples, moving-average loss 0.04
6,602,500 examples, moving-average loss 0.04
6,603,125 examples, moving-average loss 0.04
6,603,750 examples, moving-average loss 0.04
6,604,375 examples, moving-average loss 0.04
6,605,000 examples, moving-average loss 0.04
6,605,625 

In [22]:
import models; reload(models)

# Specify model hyperparameters as used by model_fn
model_params = dict(V=vocab.size, embed_dim=50, hidden_dims=[25], num_classes=2,
                    encoder_type='bow',
                    lr=0.1, optimizer='adagrad', beta=0.01)

checkpoint_dir = "/tmp/tf_bow_sst_" + datetime.datetime.now().strftime("%Y%m%d-%H%M")
if os.path.isdir(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
# Write vocabulary to file, so TensorBoard can label embeddings.
# creates checkpoint_dir/projector_config.pbtxt and checkpoint_dir/metadata.tsv
vocab.write_projector_config(checkpoint_dir, "Encoder/Embedding_Layer/W_embed")

model = tf.estimator.Estimator(model_fn=models.classifier_model_fn, 
                               params=model_params,
                               model_dir=checkpoint_dir)
print("")
print("To view training (once it starts), run:\n")
print("    tensorboard --logdir='{:s}' --port 6006".format(checkpoint_dir))
print("\nThen in your browser, open: http://localhost:6006")

Vocabulary (5,681 words) written to '/tmp/tf_bow_sst_20181202-1556/metadata.tsv'
Projector config written to /tmp/tf_bow_sst_20181202-1556/projector_config.pbtxt
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tf_bow_sst_20181202-1556', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77350183c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}

To view training (once it starts), run:

    tensorboard --logdir='/tmp/tf_bow_sst_20181202-1556' --port 6006

Then in your browser, open: http:

In [ ]:
# Training params, just used in this cell for the input_fn-s
train_params = dict(batch_size=25, total_epochs=10, eval_every=2)
assert(train_params['total_epochs'] % train_params['eval_every'] == 0)

# Construct and train the model, saving checkpoints to the directory above.
# Input function for training set batches
# Do 'eval_every' epochs at once, followed by evaluating on the dev set.
# NOTE: use patch_numpy_io.numpy_input_fn instead of tf.estimator.inputs.numpy_input_fn
train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": train_x, "ns": train_n}, y=train_y,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'], shuffle=True, seed=42
                 )

# Input function for dev set batches. As above, but:
# - Don't randomize order
# - Iterate exactly once (one epoch)
dev_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": dev_x, "ns": dev_n}, y=dev_y,
                    batch_size=25, num_epochs=1, shuffle=False
                )

for _ in range(train_params['total_epochs'] // train_params['eval_every']):
    # Train for a few epochs, then evaluate on dev
    model.train(input_fn=train_input_fn)
    eval_metrics = model.evaluate(input_fn=dev_input_fn, name="dev")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tf_bow_sst_20181202-1556/model.ckpt.
INFO:tensorflow:loss = 1.1977599, step = 1
INFO:tensorflow:global_step/sec: 219.809
INFO:tensorflow:loss = 0.3024713, step = 101 (0.457 sec)
INFO:tensorflow:global_step/sec: 248.69
INFO:tensorflow:loss = 0.1790441, step = 201 (0.402 sec)
INFO:tensorflow:global_step/sec: 243.634
INFO:tensorflow:loss = 0.117911056, step = 301 (0.411 sec)
INFO:tensorflow:global_step/sec: 247.446
INFO:tensorflow:loss = 0.08458439, step = 401 (0.404 sec)
INFO:tensorflow:global_step/sec: 247.608
INFO:tensorflow:loss = 0.06527513, step = 501 (0.405 sec)
INFO:tensorflow:global_step/sec: 245.239
INFO:tensorflow:loss = 0.054522153, step = 601 (0.406 sec)
INFO:tensorflow:global_step/sec: 242.11

INFO:tensorflow:global_step/sec: 236.64
INFO:tensorflow:loss = 0.039720543, step = 7901 (0.423 sec)
INFO:tensorflow:global_step/sec: 231.461
INFO:tensorflow:loss = 0.039437592, step = 8001 (0.430 sec)
INFO:tensorflow:global_step/sec: 239.414
INFO:tensorflow:loss = 0.039200913, step = 8101 (0.419 sec)
INFO:tensorflow:global_step/sec: 241.446
INFO:tensorflow:loss = 0.6392902, step = 8201 (0.414 sec)
INFO:tensorflow:global_step/sec: 243.692
INFO:tensorflow:loss = 0.06104687, step = 8301 (0.410 sec)
INFO:tensorflow:global_step/sec: 234.149
INFO:tensorflow:loss = 0.04208561, step = 8401 (0.428 sec)
INFO:tensorflow:global_step/sec: 238.789
INFO:tensorflow:loss = 0.04038766, step = 8501 (0.419 sec)
INFO:tensorflow:global_step/sec: 240.131
INFO:tensorflow:loss = 0.039849415, step = 8601 (0.416 sec)
INFO:tensorflow:global_step/sec: 234.585
INFO:tensorflow:loss = 0.039300926, step = 8701 (0.427 sec)
INFO:tensorflow:global_step/sec: 243.209
INFO:tensorflow:loss = 0.039150123, step = 8801 (0.411 s

INFO:tensorflow:loss = 0.03401563, step = 16001 (0.440 sec)
INFO:tensorflow:global_step/sec: 238.77
INFO:tensorflow:loss = 0.03509404, step = 16101 (0.418 sec)
INFO:tensorflow:global_step/sec: 237.647
INFO:tensorflow:loss = 0.036174424, step = 16201 (0.420 sec)
INFO:tensorflow:global_step/sec: 235.912
INFO:tensorflow:loss = 0.035442952, step = 16301 (0.427 sec)
INFO:tensorflow:global_step/sec: 228.21
INFO:tensorflow:loss = 0.03709129, step = 16401 (0.436 sec)
INFO:tensorflow:global_step/sec: 231.331
INFO:tensorflow:loss = 0.03584661, step = 16501 (0.434 sec)
INFO:tensorflow:global_step/sec: 233.965
INFO:tensorflow:loss = 0.03409261, step = 16601 (0.426 sec)
INFO:tensorflow:global_step/sec: 245.433
INFO:tensorflow:loss = 0.033030428, step = 16701 (0.408 sec)
INFO:tensorflow:global_step/sec: 247.064
INFO:tensorflow:loss = 0.03300414, step = 16801 (0.405 sec)
INFO:tensorflow:global_step/sec: 234.805
INFO:tensorflow:loss = 0.034794874, step = 16901 (0.426 sec)
INFO:tensorflow:global_step/s

INFO:tensorflow:loss = 0.050526768, step = 24101 (0.434 sec)
INFO:tensorflow:global_step/sec: 238.116
INFO:tensorflow:loss = 0.046627905, step = 24201 (0.420 sec)
INFO:tensorflow:global_step/sec: 235.258
INFO:tensorflow:loss = 0.048294585, step = 24301 (0.424 sec)
INFO:tensorflow:global_step/sec: 223.999
INFO:tensorflow:loss = 0.047609456, step = 24401 (0.447 sec)
INFO:tensorflow:global_step/sec: 233.264
INFO:tensorflow:loss = 0.04452582, step = 24501 (0.429 sec)
INFO:tensorflow:global_step/sec: 232.3
INFO:tensorflow:loss = 0.043430585, step = 24601 (0.429 sec)
INFO:tensorflow:global_step/sec: 233.324
INFO:tensorflow:loss = 0.04418249, step = 24701 (0.429 sec)
INFO:tensorflow:global_step/sec: 233.248
INFO:tensorflow:loss = 0.41107187, step = 24801 (0.429 sec)
INFO:tensorflow:global_step/sec: 227.702
INFO:tensorflow:loss = 0.07314973, step = 24901 (0.439 sec)
INFO:tensorflow:global_step/sec: 240.205
INFO:tensorflow:loss = 0.049608275, step = 25001 (0.416 sec)
INFO:tensorflow:global_step

INFO:tensorflow:loss = 0.051388633, step = 32201 (0.461 sec)
INFO:tensorflow:global_step/sec: 232.044
INFO:tensorflow:loss = 0.04855955, step = 32301 (0.431 sec)
INFO:tensorflow:global_step/sec: 226.841
INFO:tensorflow:loss = 0.049590994, step = 32401 (0.439 sec)
INFO:tensorflow:global_step/sec: 238.106
INFO:tensorflow:loss = 0.045414656, step = 32501 (0.421 sec)
INFO:tensorflow:global_step/sec: 231.926
INFO:tensorflow:loss = 0.043116897, step = 32601 (0.431 sec)
INFO:tensorflow:global_step/sec: 226.991
INFO:tensorflow:loss = 0.043856915, step = 32701 (0.441 sec)
INFO:tensorflow:global_step/sec: 226.501
INFO:tensorflow:loss = 0.044738203, step = 32801 (0.441 sec)
INFO:tensorflow:global_step/sec: 230.188
INFO:tensorflow:loss = 0.3870337, step = 32901 (0.434 sec)
INFO:tensorflow:global_step/sec: 201.395
INFO:tensorflow:loss = 0.049539283, step = 33001 (0.500 sec)
INFO:tensorflow:global_step/sec: 152.447
INFO:tensorflow:loss = 0.04210788, step = 33101 (0.660 sec)
INFO:tensorflow:global_st

...

INFO:tensorflow:loss = 0.068921804, step = 409901 (0.426 sec)
INFO:tensorflow:global_step/sec: 235.891
INFO:tensorflow:loss = 0.06210369, step = 410001 (0.423 sec)
INFO:tensorflow:global_step/sec: 237.285
INFO:tensorflow:loss = 0.06354676, step = 410101 (0.421 sec)
INFO:tensorflow:global_step/sec: 237.784
INFO:tensorflow:loss = 0.059733115, step = 410201 (0.421 sec)
INFO:tensorflow:global_step/sec: 232.859
INFO:tensorflow:loss = 0.059924062, step = 410301 (0.430 sec)
INFO:tensorflow:global_step/sec: 233.032
INFO:tensorflow:loss = 0.059727672, step = 410401 (0.428 sec)
INFO:tensorflow:global_step/sec: 231.71
INFO:tensorflow:loss = 0.057603136, step = 410501 (0.432 sec)
INFO:tensorflow:global_step/sec: 239.961
INFO:tensorflow:loss = 0.06266952, step = 410601 (0.416 sec)
INFO:tensorflow:global_step/sec: 229.385
INFO:tensorflow:loss = 0.06651995, step = 410701 (0.436 sec)
INFO:tensorflow:global_step/sec: 235.741
INFO:tensorflow:loss = 0.056642085, step = 410801 (0.425 sec)
INFO:tensorflow:

INFO:tensorflow:global_step/sec: 232.455
INFO:tensorflow:loss = 0.04060665, step = 417901 (0.430 sec)
INFO:tensorflow:global_step/sec: 240.987
INFO:tensorflow:loss = 0.03681622, step = 418001 (0.415 sec)
INFO:tensorflow:global_step/sec: 228.003
INFO:tensorflow:loss = 0.03681215, step = 418101 (0.438 sec)
INFO:tensorflow:global_step/sec: 233.845
INFO:tensorflow:loss = 0.039347656, step = 418201 (0.428 sec)
INFO:tensorflow:global_step/sec: 222.562
INFO:tensorflow:loss = 0.038486887, step = 418301 (0.449 sec)
INFO:tensorflow:global_step/sec: 230.504
INFO:tensorflow:loss = 0.0380228, step = 418401 (0.434 sec)
INFO:tensorflow:global_step/sec: 232.348
INFO:tensorflow:loss = 0.03857761, step = 418501 (0.430 sec)
INFO:tensorflow:global_step/sec: 239.941
INFO:tensorflow:loss = 0.038061127, step = 418601 (0.416 sec)
INFO:tensorflow:global_step/sec: 240.631
INFO:tensorflow:loss = 0.037826058, step = 418701 (0.417 sec)
INFO:tensorflow:global_step/sec: 227.46
INFO:tensorflow:loss = 0.04006406, step

INFO:tensorflow:global_step/sec: 236.008
INFO:tensorflow:loss = 0.039221212, step = 425901 (0.424 sec)
INFO:tensorflow:global_step/sec: 231.418
INFO:tensorflow:loss = 0.039355192, step = 426001 (0.433 sec)
INFO:tensorflow:global_step/sec: 229.802
INFO:tensorflow:loss = 0.040156934, step = 426101 (0.435 sec)
INFO:tensorflow:global_step/sec: 237.601
INFO:tensorflow:loss = 0.039099306, step = 426201 (0.420 sec)
INFO:tensorflow:global_step/sec: 236.94
INFO:tensorflow:loss = 0.04003782, step = 426301 (0.423 sec)
INFO:tensorflow:global_step/sec: 243.666
INFO:tensorflow:loss = 0.03942547, step = 426401 (0.410 sec)
INFO:tensorflow:global_step/sec: 238.968
INFO:tensorflow:loss = 0.039595887, step = 426501 (0.419 sec)
INFO:tensorflow:global_step/sec: 240.672
INFO:tensorflow:loss = 0.43236247, step = 426601 (0.416 sec)
INFO:tensorflow:global_step/sec: 246.091
INFO:tensorflow:loss = 0.06430468, step = 426701 (0.406 sec)
INFO:tensorflow:global_step/sec: 248.126
INFO:tensorflow:loss = 0.05517516, st

INFO:tensorflow:global_step/sec: 231.028
INFO:tensorflow:loss = 0.041659247, step = 433901 (0.433 sec)
INFO:tensorflow:global_step/sec: 237.281
INFO:tensorflow:loss = 0.04152543, step = 434001 (0.421 sec)
INFO:tensorflow:global_step/sec: 238.887
INFO:tensorflow:loss = 0.040909473, step = 434101 (0.418 sec)
INFO:tensorflow:global_step/sec: 234.372
INFO:tensorflow:loss = 0.04065641, step = 434201 (0.428 sec)
INFO:tensorflow:global_step/sec: 238.965
INFO:tensorflow:loss = 0.040848028, step = 434301 (0.417 sec)
INFO:tensorflow:global_step/sec: 235.328
INFO:tensorflow:loss = 0.04167241, step = 434401 (0.425 sec)
INFO:tensorflow:global_step/sec: 239.93
INFO:tensorflow:loss = 0.040947955, step = 434501 (0.417 sec)
INFO:tensorflow:global_step/sec: 240.353
INFO:tensorflow:loss = 0.040813923, step = 434601 (0.416 sec)
INFO:tensorflow:global_step/sec: 241.643
INFO:tensorflow:loss = 0.03995047, step = 434701 (0.414 sec)
INFO:tensorflow:global_step/sec: 240.687
INFO:tensorflow:loss = 0.039611056, s

INFO:tensorflow:loss = 0.034801953, step = 441901 (0.429 sec)
INFO:tensorflow:global_step/sec: 231.247
INFO:tensorflow:loss = 0.034117714, step = 442001 (0.432 sec)
INFO:tensorflow:global_step/sec: 232.947
INFO:tensorflow:loss = 0.035551958, step = 442101 (0.429 sec)
INFO:tensorflow:global_step/sec: 236.974
INFO:tensorflow:loss = 0.036341336, step = 442201 (0.422 sec)
INFO:tensorflow:global_step/sec: 235.803
INFO:tensorflow:loss = 0.034905843, step = 442301 (0.423 sec)
INFO:tensorflow:global_step/sec: 241.202
INFO:tensorflow:loss = 0.03408986, step = 442401 (0.416 sec)
INFO:tensorflow:global_step/sec: 232.914
INFO:tensorflow:loss = 0.034757778, step = 442501 (0.432 sec)
INFO:tensorflow:global_step/sec: 229.547
INFO:tensorflow:loss = 0.034300923, step = 442601 (0.433 sec)
INFO:tensorflow:global_step/sec: 231.497
INFO:tensorflow:loss = 0.03441185, step = 442701 (0.432 sec)
INFO:tensorflow:global_step/sec: 238.265
INFO:tensorflow:loss = 0.03405151, step = 442801 (0.420 sec)
INFO:tensorflo

In [ ]:

# test_input_fn = None  # replace with an input_fn, similar to dev_input_fn

test_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": test_x, "ns": test_n}, y=test_y,
                    batch_size=128, num_epochs=1, shuffle=False
                )

eval_metrics = model.evaluate(input_fn = test_input_fn)  # replace with result of model.evaluate(...)

#### END(YOUR CODE) ####
print("Accuracy on test set: {:.02%}".format(eval_metrics['accuracy']))
eval_metrics

In [ ]:
from sklearn.metrics import accuracy_score
predictions = list(model.predict(test_input_fn))  # list of dicts
y_pred = [p['max'] for p in predictions]
acc = accuracy_score(y_pred, test_y)

print("Accuracy on test set: {:.02%}".format(acc))

In [ ]:
# predicted labels
print(len(y_pred))
print(predictions[:2])